In [5]:
# Importing essential libraries

# Data Gathering 
import pandas as pd
import numpy as np

# Data Visualization
import seaborn as sns



#### Data Gathering 

In [7]:
deliveries = pd.read_csv('deliveries.csv')
deliveries.sample(7)

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
147711,1082639,1,Kings XI Punjab,Kolkata Knight Riders,1,3,MJ Guptill,C de Grandhomme,M Vohra,0,0,0,NaN,0,NaN,NaN,NaN
90854,598058,1,Chennai Super Kings,Rajasthan Royals,5,5,M Vijay,SR Watson,MEK Hussey,2,0,2,NaN,0,NaN,NaN,NaN
194216,1254060,1,Kolkata Knight Riders,Sunrisers Hyderabad,18,2,Shakib Al Hasan,T Natarajan,KD Karthik,1,0,1,NaN,0,NaN,NaN,NaN
88014,598046,1,Mumbai Indians,Chennai Super Kings,12,3,AT Rayudu,B Laughlin,RG Sharma,2,0,2,NaN,0,NaN,NaN,NaN
257994,1426297,2,Chennai Super Kings,Gujarat Titans,2,5,RD Gaikwad,UT Yadav,DJ Mitchell,0,0,0,NaN,1,RD Gaikwad,caught,Rashid Khan
54347,501252,2,Royal Challengers Bangalore,Rajasthan Royals,14,4,CH Gayle,SK Warne,V Kohli,0,0,0,NaN,0,NaN,NaN,NaN
44828,501211,2,Royal Challengers Bangalore,Chennai Super Kings,2,6,MA Agarwal,JA Morkel,V Kohli,0,0,0,NaN,0,NaN,NaN,NaN


`Column` | Description
:-|:-
`match_id` |	Unique identifier for each match, used to link with match details.
`inning`|	Indicates the innings (1 or 2) in the match.
`batting_team`|Name of the team batting in the given inning.
`bowling_team`|	Name of the team bowling in the given inning.
`over`|	The over number in the current inning (e.g., 1, 2, ..., 20).
`ball`|	The ball number within the over (1 to 6).
`batter`|	Name of the player currently batting.
`bowler`	|Name of the player currently bowling.
`non_striker`|	Name of the non-striker at the other end of the pitch.
`batsman_runs`|	Number of runs scored by the batsman on the current delivery (can include 0 for dot balls or 4/6 for boundaries).
`extra_runs`	|Number of extra runs (such as no-balls, wides) on this delivery.
`total_runs`	|Total runs scored on this ball (sum of batsman runs and extra runs).
`extras_type`	|Type of extra run, if applicable (e.g., wide, no-ball, bye, legbye).
`is_wicket`	|Indicates if the delivery resulted in a wicket (1 for wicket, 0 for no wicket).
`player_dismissed`|	Name of the player dismissed on the ball (if applicable, otherwise null).
`dismissal_kind`	|Type of dismissal (e.g., bowled, caught, run out, LBW).
`fielder`	|Name of the fielder involved in the dismissal (for dismissals like catch, run out, etc.; null for dismissals not involving a fielder).

In [9]:
matches = pd.read_csv('matches.csv')
matches.sample(7)

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
305,548365,2012,Jaipur,2012-05-13,League,A Chandila,Sawai Mansingh Stadium,Rajasthan Royals,Pune Warriors,Rajasthan Royals,bat,Rajasthan Royals,runs,45.0,171.0,20.0,N,NaN,BF Bowden,SK Tarapore
360,598035,2013,Raipur,2013-04-28,League,DA Warner,Shaheed Veer Narayan Singh International Stadium,Delhi Daredevils,Pune Warriors,Pune Warriors,field,Delhi Daredevils,runs,15.0,165.0,20.0,N,NaN,CK Nandan,S Ravi
14,335996,2007/08,Bangalore,2008-04-28,League,MS Dhoni,M Chinnaswamy Stadium,Royal Challengers Bangalore,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,13.0,179.0,20.0,N,NaN,BR Doctrove,RB Tiffin
588,1082602,2017,Bangalore,2017-04-14,League,KA Pollard,M Chinnaswamy Stadium,Royal Challengers Bangalore,Mumbai Indians,Mumbai Indians,field,Mumbai Indians,wickets,4.0,143.0,20.0,N,NaN,KN Ananthapadmanabhan,AK Chaudhary
870,1254088,2021,Abu Dhabi,2021-10-08,League,Ishan Kishan,"Zayed Cricket Stadium, Abu Dhabi",Mumbai Indians,Sunrisers Hyderabad,Mumbai Indians,bat,Mumbai Indians,runs,42.0,236.0,20.0,N,NaN,Tapan Sharma,VK Sharma
164,419155,2009/10,Chennai,2010-04-15,League,G Gambhir,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Delhi Daredevils,Chennai Super Kings,bat,Delhi Daredevils,wickets,6.0,113.0,20.0,N,NaN,HDPK Dharmasena,SS Hazare
585,1082599,2017,Pune,2017-04-11,League,SV Samson,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Delhi Daredevils,Rising Pune Supergiant,field,Delhi Daredevils,runs,97.0,206.0,20.0,N,NaN,AY Dandekar,S Ravi


`Column`	|Description
:-|:-
`id`	|Unique identifier for each match, similar to match_id in the ball-by-ball dataset, useful for linking datasets.
`season`|	Year in which the IPL season took place.
`city`	|City where the match was played.
`date`	|Date of the match.
`match_type`|	Type of match (could indicate regular season, playoff, or final).
`player_of_match`|	Name of the player awarded "Player of the Match."
`venue`|	Stadium where the match was played.
`team1`	|Name of the first team (order doesn’t imply batting or bowling first).
`team2`	|Name of the second team.
`toss_winner`|	Team that won the toss.
`toss_decision`|	Decision taken by the toss-winning team (bat or field).
`winner`|Name of the team that won the match.
`result`|	Outcome type of the match (e.g., normal, tie, no result).
`result_margin`|	Margin by which the team won (e.g., runs or wickets).
`target_runs`	|Target runs set for the chasing team (null if not applicable).
`target_overs`	|Target overs if the match is shortened (e.g., in rain-affected matches; null if not applicable).
`super_over`	|Indicates if the match had a super over (1 for yes, 0 for no).
`method`	|Method used to determine the result in rain-affected games (e.g., Duckworth-Lewis-Stern, or null if not applicable).
`umpire1`	|Name of the first on-field umpire.
`umpire2`	|Name of the second on-field umpire.

- here we have two datasets one containing ball by ball details and other containing match wise details.
- we are going to analyse each data set for insights.


#### Data Cleaning

In [13]:
deliveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260920 entries, 0 to 260919
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          260920 non-null  int64 
 1   inning            260920 non-null  int64 
 2   batting_team      260920 non-null  object
 3   bowling_team      260920 non-null  object
 4   over              260920 non-null  int64 
 5   ball              260920 non-null  int64 
 6   batter            260920 non-null  object
 7   bowler            260920 non-null  object
 8   non_striker       260920 non-null  object
 9   batsman_runs      260920 non-null  int64 
 10  extra_runs        260920 non-null  int64 
 11  total_runs        260920 non-null  int64 
 12  extras_type       14125 non-null   object
 13  is_wicket         260920 non-null  int64 
 14  player_dismissed  12950 non-null   object
 15  dismissal_kind    12950 non-null   object
 16  fielder           9354 non-null    obj

- here we get data information where we know basics about our data
    - There are 260920 rows and 17 columns in deliveries dataset.
    - some columns have missing values.


In [15]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1095 non-null   int64  
 1   season           1095 non-null   object 
 2   city             1044 non-null   object 
 3   date             1095 non-null   object 
 4   match_type       1095 non-null   object 
 5   player_of_match  1090 non-null   object 
 6   venue            1095 non-null   object 
 7   team1            1095 non-null   object 
 8   team2            1095 non-null   object 
 9   toss_winner      1095 non-null   object 
 10  toss_decision    1095 non-null   object 
 11  winner           1090 non-null   object 
 12  result           1095 non-null   object 
 13  result_margin    1076 non-null   float64
 14  target_runs      1092 non-null   float64
 15  target_overs     1092 non-null   float64
 16  super_over       1095 non-null   object 
 17  method        

- Here  we have 1095 rows and 20 columns .
- there are some missing values in this data.
- Date column is in form of object which can be converted into date time for better analysis.

##### Further I cleaned both datas column wise 

#### Deliveries cleaning 

In [19]:
deliveries.sample(25)

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
51659,501241,1,Mumbai Indians,Pune Warriors,12,2,RG Sharma,Yuvraj Singh,TL Suman,0,0,0,NaN,0,NaN,NaN,NaN
260005,1426307,2,Sunrisers Hyderabad,Punjab Kings,14,3,Shahbaz Ahmed,Arshdeep Singh,H Klaasen,1,0,1,NaN,0,NaN,NaN,NaN
172060,1178406,1,Royal Challengers Bangalore,Mumbai Indians,5,3,AB de Villiers,JJ Bumrah,PA Patel,0,0,0,NaN,0,NaN,NaN,NaN
224567,1304115,1,Delhi Capitals,Mumbai Indians,9,4,R Powell,HR Shokeen,RR Pant,1,0,1,NaN,0,NaN,NaN,NaN
224277,1304114,1,Chennai Super Kings,Rajasthan Royals,2,3,DP Conway,TA Boult,MM Ali,6,0,6,NaN,0,NaN,NaN,NaN
118858,829791,1,Kings XI Punjab,Kolkata Knight Riders,18,1,Gurkeerat Singh,UT Yadav,DA Miller,1,0,1,NaN,0,NaN,NaN,NaN
189331,1216534,1,Royal Challengers Bangalore,Sunrisers Hyderabad,12,3,AB de Villiers,Rashid Khan,V Kohli,1,0,1,NaN,0,NaN,NaN,NaN
53492,501249,1,Chennai Super Kings,Rajasthan Royals,6,6,M Vijay,SK Warne,MEK Hussey,1,0,1,NaN,0,NaN,NaN,NaN
56959,501264,2,Kings XI Punjab,Deccan Chargers,1,2,PC Valthaty,DW Steyn,AC Gilchrist,0,0,0,NaN,0,NaN,NaN,NaN
49384,501231,2,Rajasthan Royals,Mumbai Indians,11,5,LRPL Taylor,SL Malinga,J Botha,0,0,0,NaN,0,NaN,NaN,NaN


#### match_id

In [21]:
deliveries['match_id'].isnull().sum()

0

In [22]:
deliveries['match_id'].nunique()

1095

- In match_id column there isn't any null value .
- Their are 1095 unique match id's which matches #rows in matches dataset.
   i.e. We have information of all matches.

#### inning

In [25]:
deliveries['inning'].isnull().sum()

0

In [26]:
deliveries['inning'].unique()

array([1, 2, 3, 4, 5, 6], dtype=int64)

- here data does not have null values.
- but innings values are little disturbed, as we know their should be only 2 innings but have till 6.
    - 3rd and 4th inning is their due to a super over but 5th and 6th inning is a entry error.

In [28]:
# If inning are 5 changing to 3 and if 6 changed to 4
def inning_replacing(inning):
    if inning == 5:
        return 3
    elif inning == 6:
        return 4
    else:
        return inning

In [29]:
deliveries['inning']=deliveries['inning'].apply(inning_replacing)

In [30]:
deliveries['inning'].unique()

array([1, 2, 3, 4], dtype=int64)

#### batting team

In [32]:
deliveries['batting_team'].isnull().sum()

0

In [33]:
deliveries['batting_team'].nunique()

19

In [34]:
deliveries['batting_team'].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Mumbai Indians', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

- their is no null value in batting team
- over the years names of teams are changed so same team is present with several names or their are some speeling difference

In [36]:
# correcting teams name if differs
def change_team_name(team_name):
    if team_name in ['Rising Pune Supergiants','Pune Warriors']:
        return 'Rising Pune Supergiant'
    elif team_name == 'Royal Challengers Bengaluru':
        return 'Royal Challengers Bangalore'
    elif team_name == 'Deccan Chargers':
        return 'Sunrisers Hyderabad'
    elif team_name == 'Delhi Daredevils':
        return 'Delhi Capitals'
    elif team_name == 'Gujarat Lions':
        return 'Gujarat Titans'
    elif team_name == 'Kings XI Punjab':
        return 'Punjab Kings'
    else:
        return team_name

In [37]:
deliveries['batting_team'] = deliveries['batting_team'].apply(change_team_name)

In [38]:
deliveries['batting_team'].nunique()

12

- there were 19 teams in batting team column which reduced to 12 as we know till today 12 teams were played in IPL

#### bowling team

In [41]:
deliveries['bowling_team'].isnull().sum()

0

In [42]:
deliveries['bowling_team'].nunique()

19

In [43]:
deliveries['bowling_team'] = deliveries['bowling_team'].apply(change_team_name)

In [44]:
deliveries['bowling_team'].nunique()

12

- their were 19 teams reduced to 12 with same problem as batting team

#### Over

In [47]:
deliveries['over'].isnull().sum()

0

In [48]:
deliveries['over'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int64)

- 20 overs are present 1 over is calculated after completing.

#### ball

In [51]:
deliveries['ball'].isnull().sum()

0

In [52]:
deliveries['ball'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64)

- as we know each over has 6 balls their can be wides and no balls in some overs.

#### batter

In [55]:
deliveries['batter'].isnull().sum()

0

In [56]:
deliveries['batter'].nunique()

673

In [57]:
deliveries['batter'].unique()

array(['SC Ganguly', 'BB McCullum', 'RT Ponting', 'DJ Hussey',
       'Mohammad Hafeez', 'R Dravid', 'W Jaffer', 'V Kohli', 'JH Kallis',
       'CL White', 'MV Boucher', 'B Akhil', 'AA Noffke', 'P Kumar',
       'Z Khan', 'SB Joshi', 'PA Patel', 'ML Hayden', 'MEK Hussey',
       'MS Dhoni', 'SK Raina', 'JDP Oram', 'S Badrinath', 'K Goel',
       'JR Hopes', 'KC Sangakkara', 'Yuvraj Singh', 'SM Katich',
       'IK Pathan', 'T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif',
       'DS Lehmann', 'RA Jadeja', 'M Rawat', 'D Salunkhe', 'SK Warne',
       'SK Trivedi', 'G Gambhir', 'V Sehwag', 'S Dhawan', 'L Ronchi',
       'ST Jayasuriya', 'DJ Thornely', 'RV Uthappa', 'PR Shah',
       'AM Nayar', 'SM Pollock', 'Harbhajan Singh', 'S Chanderpaul',
       'LRPL Taylor', 'AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxman',
       'A Symonds', 'RG Sharma', 'SB Styris', 'AS Yadav', 'SB Bangar',
       'WPUJC Vaas', 'RP Singh', 'WP Saha', 'LR Shukla',
       'DPMD Jayawardene', 'S Sohal', 'B Lee', 'PP Cha

- there are total 673 batter played in IPL till now

#### bowler

In [60]:
deliveries['bowler'].isnull().sum()

0

In [61]:
deliveries['bowler'].nunique()

530

In [62]:
deliveries['bowler'].unique()

array(['P Kumar', 'Z Khan', 'AA Noffke', 'JH Kallis', 'SB Joshi',
       'CL White', 'AB Dinda', 'I Sharma', 'AB Agarkar', 'SC Ganguly',
       'LR Shukla', 'B Lee', 'S Sreesanth', 'JR Hopes', 'IK Pathan',
       'K Goel', 'PP Chawla', 'WA Mota', 'JDP Oram', 'MS Gony',
       'M Muralitharan', 'P Amarnath', 'Joginder Sharma', 'GD McGrath',
       'B Geeves', 'MF Maharoof', 'R Bhatia', 'DL Vettori', 'MM Patel',
       'SR Watson', 'SK Trivedi', 'SK Warne', 'YK Pathan', 'D Salunkhe',
       'R Vinay Kumar', 'B Akhil', 'A Nehra', 'SM Pollock', 'DS Kulkarni',
       'ST Jayasuriya', 'Harbhajan Singh', 'AM Nayar', 'M Kartik',
       'Mohammad Hafeez', 'DJ Hussey', 'WPUJC Vaas', 'RP Singh',
       'SB Styris', 'SB Bangar', 'A Symonds', 'PP Ojha', 'Pankaj Singh',
       'Mohammad Asif', 'VY Mahesh', 'Shahid Afridi', 'DJ Bravo',
       'VS Yeligati', 'MA Khote', 'D Kalyankrishna', 'VRV Singh',
       'Sohail Tanvir', 'A Kumble', 'DNT Zoysa', 'SD Chitnis',
       'Yuvraj Singh', 'Shoaib Malik',

- 530 bowlers played IPL till now

#### non-striker

In [65]:
deliveries['non_striker'].isnull().sum()

0

In [66]:
deliveries['non_striker'].nunique()

663

In [67]:
deliveries['non_striker'].unique()

array(['BB McCullum', 'SC Ganguly', 'RT Ponting', 'DJ Hussey',
       'Mohammad Hafeez', 'W Jaffer', 'R Dravid', 'V Kohli', 'JH Kallis',
       'CL White', 'MV Boucher', 'B Akhil', 'AA Noffke', 'P Kumar',
       'Z Khan', 'SB Joshi', 'ML Hayden', 'PA Patel', 'MEK Hussey',
       'MS Dhoni', 'SK Raina', 'JDP Oram', 'S Badrinath', 'JR Hopes',
       'K Goel', 'KC Sangakkara', 'Yuvraj Singh', 'SM Katich',
       'IK Pathan', 'YK Pathan', 'T Kohli', 'SR Watson', 'M Kaif',
       'DS Lehmann', 'RA Jadeja', 'M Rawat', 'D Salunkhe', 'SK Warne',
       'SK Trivedi', 'V Sehwag', 'G Gambhir', 'S Dhawan', 'ST Jayasuriya',
       'L Ronchi', 'DJ Thornely', 'RV Uthappa', 'PR Shah', 'AM Nayar',
       'SM Pollock', 'Harbhajan Singh', 'S Chanderpaul', 'LRPL Taylor',
       'Y Venugopal Rao', 'AC Gilchrist', 'VVS Laxman', 'A Symonds',
       'SB Styris', 'AS Yadav', 'SB Bangar', 'WPUJC Vaas', 'RP Singh',
       'PP Ojha', 'WP Saha', 'LR Shukla', 'DPMD Jayawardene', 'S Sohal',
       'PP Chawla', 'WA M

- there are total 663 unique values in non-striker

#### batsmam_runs

In [70]:
deliveries['batsman_runs'].isnull().sum()

0

In [71]:
deliveries['batsman_runs'].unique()

array([0, 4, 6, 1, 2, 5, 3], dtype=int64)

- values are till 6 that a batter can strike

#### extra_runs

In [74]:
deliveries['extra_runs'].isnull().sum()

0

In [75]:
deliveries['extra_runs'].unique()

array([1, 0, 5, 4, 2, 3, 7], dtype=int64)

- because of bowlers mistake some penalties are gives so the value is 7

#### Total_runs

In [119]:
deliveries['total_runs'].isnull().sum()

0

In [121]:
deliveries['total_runs'].unique()

array([1, 0, 4, 6, 5, 2, 3, 7], dtype=int64)

- there is no anamolous behaviour in this column

#### extras_type

In [129]:
deliveries['extras_type'].isnull().sum()

246795

In [137]:
deliveries['extras_type'].unique()

array(['legbyes', nan, 'wides', 'byes', 'noballs', 'penalty'],
      dtype=object)

- here we have 246795 null values which is valid as their isnt any extra runs for these ball.
- there is no need to fill these null values 
- we can futher drop this column as it is of no use.

#### is_wicket

In [141]:
deliveries['is_wicket'].isnull().sum()

0

In [143]:
deliveries['is_wicket'].unique()

array([0, 1], dtype=int64)

- in this column 0 means no wicket and 1 mean they caught a wicket.


#### player_dismissed

In [147]:
deliveries['player_dismissed'].isnull().sum()

247970

In [149]:
deliveries['player_dismissed'].unique()

array([nan, 'SC Ganguly', 'RT Ponting', 'DJ Hussey', 'R Dravid',
       'V Kohli', 'JH Kallis', 'W Jaffer', 'MV Boucher', 'B Akhil',
       'CL White', 'AA Noffke', 'Z Khan', 'SB Joshi', 'PA Patel',
       'ML Hayden', 'MS Dhoni', 'SK Raina', 'JDP Oram', 'K Goel',
       'JR Hopes', 'Yuvraj Singh', 'KC Sangakkara', 'T Kohli',
       'YK Pathan', 'SR Watson', 'DS Lehmann', 'M Kaif', 'M Rawat',
       'RA Jadeja', 'SK Warne', 'V Sehwag', 'L Ronchi', 'DJ Thornely',
       'ST Jayasuriya', 'PR Shah', 'RV Uthappa', 'AM Nayar', 'SM Pollock',
       'S Chanderpaul', 'LRPL Taylor', 'Y Venugopal Rao', 'VVS Laxman',
       'AC Gilchrist', 'RG Sharma', 'SB Styris', 'AS Yadav', 'A Symonds',
       'WPUJC Vaas', 'SB Bangar', 'PP Ojha', 'BB McCullum', 'WP Saha',
       'Mohammad Hafeez', 'DPMD Jayawardene', 'IK Pathan', 'B Lee',
       'S Sohal', 'Kamran Akmal', 'Shahid Afridi', 'G Gambhir',
       'MEK Hussey', 'DJ Bravo', 'MA Khote', 'Harbhajan Singh',
       'GC Smith', 'D Salunkhe', 'RR Sarwan',

- in this column we have the player of whose the wicket is caught
- the data is of every ball so their are many null value which is valid. 
- the column does not gives any specific value so we might delete it later.

#### dismissal_kind

In [153]:
deliveries['dismissal_kind'].isnull().sum()

247970

In [155]:
deliveries['dismissal_kind'].unique()

array([nan, 'caught', 'bowled', 'run out', 'lbw', 'retired hurt',
       'stumped', 'caught and bowled', 'hit wicket',
       'obstructing the field', 'retired out'], dtype=object)

- here also null values are valid due to same reason

#### fielder

In [161]:
deliveries['fielder'].isnull().sum()

251566

In [165]:
deliveries['fielder'].unique()

array([nan, 'JH Kallis', 'P Kumar', 'CL White', 'M Kartik', 'RT Ponting',
       'WP Saha', 'AB Agarkar', 'BB McCullum', 'KC Sangakkara',
       'IK Pathan', 'PA Patel', 'Joginder Sharma', 'P Amarnath',
       'M Manhas', 'B Geeves', 'V Kohli', 'Z Khan', 'MV Boucher',
       'AM Nayar', 'L Ronchi', 'LR Shukla', 'Mohammad Hafeez', 'AB Dinda',
       'DJ Hussey', 'SC Ganguly', 'RG Sharma', 'SB Styris', 'A Symonds',
       'Kamran Akmal', 'RA Jadeja', 'PP Chawla', 'S Dhawan', 'KD Karthik',
       'MK Tiwary', 'GD McGrath', 'G Gambhir', 'AC Gilchrist', 'DJ Bravo',
       'Harbhajan Singh', 'S Badrinath', 'JDP Oram', 'SK Raina',
       'D Salunkhe', 'SB Bangar', 'D Kalyankrishna', 'VVS Laxman',
       'DS Kulkarni', 'AM Rahane', 'A Nehra', 'B Lee', 'RR Sarwan',
       'Yuvraj Singh', 'MM Patel', 'GC Smith', 'M Rawat', 'Sohail Tanvir',
       'A Kumble', 'M Muralitharan', 'MS Dhoni', 'I Sharma', 'PP Ojha',
       'DPMD Jayawardene', 'V Sehwag', 'W Jaffer', 'R Vinay Kumar',
       'R Dravid',

- fielders who caused the dismissal of the player
- therefore null values are valid.